In [469]:
import cobra
import pandas as pd
import io, re
from bioservices.uniprot import UniProt
import bioservices.xmltools

import sys
sys.path.append('/home/hvdinh16/Workspace/workpy3/common/')
from custom_functions import *

import lxml
from lxml import etree
from lxml.etree import ElementTree

In [ ]:
def parse_bioservices_uniprot_XML():
    

In [2]:
model = cobra.io.load_json_model('./gsmodel/iRhtoC_r2_2.json')

Academic license - for non-commercial use only


In [3]:
[g.id for g in model.genes if g.id[:2] != 'rt']

['SPONT', 'TRUE', 'UNKNOWN']

In [4]:
idx = [g.id for g in model.genes if g not in ['UNKNOWN', 'SPONT', 'TRUE']]
cols = ['id', 'name', 'sc_id', 'uniprot', 'uniprot_name', 'uniprot_subunit', 'uniprot_subloc', 'check']
df_info = pd.DataFrame(index=idx, columns=cols)
df_info['id'] = idx

for gid in df_info.index:
    g = model.genes.get_by_id(gid)
    if 'rt' not in g.name:
        df_info.loc[g.id, 'name'] = g.name
    
    if 'sc_id' in g.annotation.keys():
        df_info.loc[g.id, 'sc_id'] = g.annotation['sc_id']
        
uniprot = UniProt(verbose=True)
for i in df_info.index:
    if pd.isnull(df_info.sc_id[i]):
        continue
        
    if ',' not in df_info.sc_id[i]:
        sc_id = df_info.sc_id[i]
    else:
        sc_id = df_info.sc_id[i].split(',')[0]
    qstr = 'gene:' + df_info.sc_id[i] + ' AND taxonomy:4932 AND reviewed:yes'
    
    search = uniprot.search(qstr)
    if search == '':
        continue
    df_search = pd.read_table(io.StringIO(search))
    
    uniprot_id = df_search.Entry[0]
    retrieve = uniprot.retrieve(uniprot_id, frmt='txt')
    
    lines = retrieve.split('\n')
    
    df_info.loc[i, 'uniprot'] = uniprot_id
    df_info.loc[i, 'uniprot_name'] = re.split('\s+', lines[0])[1]
    
    subunit = [l for l in lines if l[:2] == 'CC' and 'SUBUNIT' in l]
    subunit_stat = ''
    if len(subunit) > 1:
        subunit_stat = 'check_subunit'
        print(uniprot_id, subunit)
    elif len(subunit) > 0:
        df_info.loc[i, 'uniprot_subunit'] = subunit[0].split(': ')[1][:-1]
        
    subloc = [l for l in lines if l[:2] == 'CC' and 'SUBCELLULAR LOCATION' in l]
    subloc_stat = ''
    if len(subloc) > 1:
        subloc_stat = 'check_subloc'
        print(uniprot_id, subloc)
    elif len(subloc) > 0:
        x = subloc[0].split(': ')[1]
        x = x[:-1] if x[-1] == '.' else x
        x = x.split(' {')[0] if ' {' in x else x
        df_info.loc[i, 'uniprot_subloc'] = x
        
    stats = []
    if subunit_stat != '':
        stats.append(subunit_stat)
    if subloc_stat != '':
        stats.append(subloc_stat)
    if stats != []:
        df_info.loc[i, 'check'] = ' | '.join(stats)

INFO    [bioservices:UniProt]:  Initialising UniProt service (REST)


P40825 ['CC   -!- SUBCELLULAR LOCATION: [Isoform Cytoplasmic]: Cytoplasm.', 'CC   -!- SUBCELLULAR LOCATION: [Isoform Mitochondrial]: Mitochondrion.']
P11491 ['CC   -!- SUBCELLULAR LOCATION: [Repressible alkaline phosphatase]: Vacuole', 'CC   -!- SUBCELLULAR LOCATION: [Soluble alkaline phosphatase]: Cytoplasm.']
P32796 ['CC   -!- SUBCELLULAR LOCATION: [Isoform Mitochondrial]: Mitochondrion inner', 'CC   -!- SUBCELLULAR LOCATION: [Isoform Peroxisomal]: Peroxisome.']
P17695 ['CC   -!- SUBCELLULAR LOCATION: [Isoform Cytoplasmic]: Cytoplasm', 'CC   -!- SUBCELLULAR LOCATION: [Isoform Mitochondrial]: Mitochondrion']
P38088 ['CC   -!- SUBCELLULAR LOCATION: [Isoform Cytoplasmic]: Cytoplasm.', 'CC   -!- SUBCELLULAR LOCATION: [Isoform Mitochondrial]: Mitochondrion matrix.']
P07263 ['CC   -!- SUBCELLULAR LOCATION: [Isoform Cytoplasmic]: Cytoplasm.', 'CC   -!- SUBCELLULAR LOCATION: [Isoform Mitochondrial]: Mitochondrion.']
P00724 ['CC   -!- SUBCELLULAR LOCATION: [Isoform Intracellular]: Cytoplasm.'

In [53]:
df_info[df_info.name == 'GUT2']

,id,name,sc_id,uniprot,uniprot_name,uniprot_subunit,uniprot_subloc,check
rt2671,rt2671,GUT2,YIL155C,P32191,GPDM_YEAST,NaN,Mitochondrion inner membrane,NaN


In [51]:
df_info.head()

,id,name,sc_id,uniprot,uniprot_name,uniprot_subunit,uniprot_subloc,check
rt0434,rt0434,CYC7,YEL039C,P00045,CYC7_YEAST,NaN,Mitochondrion intermembrane space,NaN
rt4960,rt4960,DLD1,YDL174C,P32891,DLD1_YEAST,NaN,Mitochondrion inner membrane,NaN
rt8275,rt8275,DLD2,YDL178W,P46681,DLD2_YEAST,Interacts with F-actin. {ECO:0000269|PubMed:15...,Mitochondrion matrix,NaN
rt5273,rt5273,BDH1,YAL060W,P39714,BDH1_YEAST,Homodimer,Cytoplasm,NaN
rt8239,rt8239,CYB2,YML054C,P00175,CYB2_YEAST,Homotetramer,Mitochondrion intermembrane space,NaN


In [ ]:
def parse_bioservices_uniprot_XML(retrieve):
    # Parse an XML entry of Uniprot extracted by bioservices
    # Only extract info for subunit, subcellular location, and cofactor
    from bioservices.uniprot import UniProt
    
    # Initiate
    info_dict = {i:None for i in ['subunit', 'subcellular location', 'cofactor']}
    
    # Parse
    uniprot_entry = retrieve.getchildren()[0]
    comments = [i for i in iter(uniprot_entry.getchildren()) if i.tag == '{http://uniprot.org/uniprot}comment']

In [177]:
uniprot_entry = retrieve.getchildren()[0]
comments = [i for i in iter(uniprot_entry.getchildren()) if i.tag == '{http://uniprot.org/uniprot}comment']

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:2 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [367]:
x = [i for i in uniprot_entry.iter(tag='{http://uniprot.org/uniprot}comment')][0]

In [368]:
x1 = x[0]

In [370]:
x.tag

'{http://uniprot.org/uniprot}comment'

In [362]:
x.attrib

{'type': 'miscellaneous'}

In [360]:
x.getchildren()[0].text

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


'Present with 1670 molecules/cell in log phase SD medium.'

In [336]:
y1.find(xmlns+'location').text

'Mitochondrion inner membrane'

In [337]:
y2.find(xmlns+'location').text

'Mitochondrion intermembrane space'

In [339]:
y2.getchildren()

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


[<Element '{http://uniprot.org/uniprot}location' at 0x7f2342894e30>]

In [269]:
x2.attrib

{'type': 'ChEBI', 'id': 'CHEBI:57692'}

In [378]:
x = [i for i in uniprot_entry.getchildren() if i.tag == xmlns+'comment']
for i in x:
    print(i.attrib)

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


{'type': 'catalytic activity'}
{'type': 'cofactor'}
{'type': 'pathway'}
{'type': 'subcellular location'}
{'type': 'miscellaneous'}
{'type': 'similarity'}


In [ ]:
etree.tostring()

In [476]:
type(lxmlET)

lxml.etree._Element

In [513]:
qstr = 'gene:' + 'YEL039C' + ' AND taxonomy:4932 AND reviewed:yes'
search = uniprot.search(qstr)

df_search = pd.read_table(io.StringIO(search))
uniprot_id = df_search.Entry[0]
retrieve = uniprot.retrieve(uniprot_id, frmt='xml')

In [514]:
lxmlET = etree.fromstring(retrieve.data.encode())
for elem in lxmlET.getiterator():
    elem.tag = etree.QName(elem.tag).localname
etree.cleanup_namespaces(lxmlET)

In [515]:
a = lxmlET.find('entry')
comments = a.findall('comment')

In [517]:
comments = a.findall('comment')
for i in comments:
    print(i.attrib)

{'type': 'function'}
{'type': 'cofactor'}
{'type': 'subcellular location'}
{'type': 'induction'}
{'type': 'miscellaneous'}
{'type': 'similarity'}
{'type': 'online information', 'name': 'Protein Spotlight'}


In [518]:
x = comments[1]

In [521]:
x.getchildren()

[<Element cofactor at 0x7f2342552b90>, <Element text at 0x7f2342552af0>]

In [522]:
y = x.getchildren()[1]

In [528]:
y.attrib

{'evidence': '7'}

In [524]:
y.text

'Binds 1 heme group covalently per subunit.'

In [512]:
y.text

'Monomer.'

In [503]:
uniprot_entry.find('name').text

'SDHX_YEAST'

In [488]:
etree.tostring(x)

b'<comment xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" type="subcellular location">\n<subcellularLocation>\n<location evidence="2">Mitochondrion inner membrane</location>\n<topology evidence="2">Peripheral membrane protein</topology>\n<orientation evidence="2">Matrix side</orientation>\n</subcellularLocation>\n</comment>\n'

In [440]:
y = x.findall('cofactor')[0]

In [444]:
y.getchildren()

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


[<Element 'name' at 0x7f2342628ad0>, <Element 'dbReference' at 0x7f2342628b90>]

In [454]:
for i in elem.iter():
    print(i, i.attrib, i.getchildren())

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:2 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


<Element 'comment' at 0x7f234257a050> {'type': 'similarity'} [<Element 'text' at 0x7f234257a0b0>]
<Element 'text' at 0x7f234257a0b0> {'evidence': '7'} []


In [484]:
a = cofactor_entry.getchildren()[0]

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:1 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [ ]:
cofactor_attrib_list
{'cofactor', 'text'}
subloc_attrib_list
{'subcellularLocation', 'text', 'molecule'}
subunit_attrib_list
{'text'}

In [485]:
a.tag

'name'

In [486]:
str({'a':'b'})

"{'a': 'b'}"

In [ ]:
for elem in uniprot_entry.iter(tag='comment'):
    
    # Extract cofactor info
    if elem.attrib['type'] == 'cofactor':
        cofactor_list = []
        for cofactor_entry in elem.getchildren():
            name = cofactor_entry.find('name')
            if name != None:
                name = name.text
            
            dbref = cofactor_entry.find('dbReference')
            if dbref != None:
                dbref = dbref.attrib['id']
                
            cofactor_list.append(str(name) + '/' + str(dbref))
            
    # Extract subcellular location
    elif elem.attrib['type'] == 'subcellular location':
        subloc_list = []
        for subloc_entry in elem.getchildren():
            loc = subloc_entry.find('location')
            if loc != None:
                loc = loc.text
                
            subloc_list.append(loc)
        continue
            
    # Extract subunit information
    elif elem.attrib['type'] == 'subunit'

In [447]:
for elem in uniprot_entry.iter(tag='comment'):
    
    # Extract cofactor info
    if elem.attrib['type'] == 'cofactor':
        cofactor_list = []
        for cofactor_entry in elem.getchildren():
            name = cofactor_entry.find('name')
            if name != None:
                name = name.text
            
            dbref = cofactor_entry.find('dbReference')
            if dbref != None:
                dbref = dbref.attrib['id']
                
            cofactor_list.append(str(name) + '/' + str(dbref))
            
    # Extract subcellular location
    elif elem.attrib['type'] == 'subcellular location':
        subloc_list = []
        for subloc_entry in elem.getchildren():
            loc = subloc_entry.find('location')
            if loc != None:
                loc = loc.text
                
            subloc_list.append(loc)
        continue
            
    # Extract subunit information
    elif elem.attrib['type'] == 'subunit'

/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:6 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/home/hvdinh16/Workspace/workpy3/lib/python3.7/site-packages/ipykernel_launcher.py:20 DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [195]:
df_info[df_info.name == 'SDH1']

,id,name,sc_id,uniprot,uniprot_name,uniprot_subunit,uniprot_subloc,check
rt7913,rt7913,SDH1,"YKL148C,YJL045W",NaN,NaN,NaN,NaN,NaN


In [504]:
qstr = 'gene:' + 'YDR226W' + ' AND taxonomy:4932 AND reviewed:yes'
search = uniprot.search(qstr)

df_search = pd.read_table(io.StringIO(search))
uniprot_id = df_search.Entry[0]
retrieve = uniprot.retrieve(uniprot_id, frmt='xml')